In [2]:
import pandas as pd
import numpy as np

In [3]:
pm10_2004 = pd.read_csv("data/mappm102004g.csv")

pm10_2004.head()

/tmp/ipykernel_2324/310841554.py:1: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  pm10_2004 = pd.read_csv("data/mappm102004g.csv")


,pm10,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,2004,NaN,NaN,NaN
1,annual mean,NaN,NaN,NaN
2,ug m-3 (gravimetric units),NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,ukgridcode,x,y,pm102004g


In [4]:
pm10_2004.head(10)

,pm10,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,2004,NaN,NaN,NaN
1,annual mean,NaN,NaN,NaN
2,ug m-3 (gravimetric units),NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,ukgridcode,x,y,pm102004g
5,54291,460500,1221500,MISSING
6,54292,461500,1221500,MISSING
7,54294,463500,1221500,MISSING
8,54979,458500,1220500,MISSING
9,54980,459500,1220500,MISSING


In [5]:
pm10_2004.drop(labels=[0,1,2,3,4], inplace=True)

pm10_2004.rename(columns={"pm10": "ukgridcode", "Unnamed: 1": "x", "Unnamed: 2": "y", "Unnamed: 3": "pm10"}, inplace=True)

pm10_2004.head()

,ukgridcode,x,y,pm10
5,54291,460500,1221500,MISSING
6,54292,461500,1221500,MISSING
7,54294,463500,1221500,MISSING
8,54979,458500,1220500,MISSING
9,54980,459500,1220500,MISSING


In [6]:
pm10_2004["pm10"] = pd.to_numeric(pm10_2004["pm10"], errors="coerce")

In [8]:
pm10_2004 = pm10_2004.astype({"x": "int", "y": "int"})

In [9]:
x_coords = np.sort(pm10_2004["x"].unique())
y_coords = np.sort(pm10_2004["y"].unique())

In [10]:
raster = pm10_2004.pivot(index="y", columns="x", values="pm10").sort_index(ascending=False)


In [11]:
import rasterio
from rasterio.transform import from_origin

In [13]:
pixel_size = 1000
transform = from_origin(x_coords.min(), y_coords.max(), pixel_size, pixel_size)

with rasterio.open(
    "data/pollution_PM10.tif",
    "w",
    driver="GTiff",
    height=raster.shape[0],
    width=raster.shape[1],
    count=1,
    dtype="float32",
    crs="EPSG:27700",
    transform=transform,
    nodata=np.nan,
) as dst:
    dst.write(raster.values.astype("float32"), 1)

In [14]:
# Now to load the LSOA and merge them together. First it would be good to load in and format all of the pollution data into the tifs I want and use fiona rather than geopandas to 
# open lsoa file. Just use pandas and then send to SQL. 
